### **Weak Lensing Testing Notebook**

Notebook for testing of weak-lensing computed profiles for galaxy clusters in DES. The following lines can be used to analyze the structure of the dataset.
```
cd /project/projectdirs/des/www/y3_cats/Y3_mastercat_03_31_20.h5
h5ls -f -r --follow-symlinks Y3_mastercat_03_31_20.h5
```

#### **Initialize Notebook and Load Catalog**

Initializes the notebook and its package dependencies. Loads Y3 master catalog.

In [2]:
import numpy as np
import treecorr
import h5py
import kmeans_radec
import seaborn as sns
import matplotlib.pyplot as plt 
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

sns.set(style='ticks')

%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'

cat_str = '/project/projectdirs/des/www/y3_cats/Y3_mastercat_03_31_20.h5'
f = h5py.File(cat_str,'r')

#### **Load and Setup Galaxy and Shear Data**

Loads the positions and redshifts (bpz) of galaxies with robust shear estimates alongside shear estimates. Creates jackknife patches.

In [3]:
### ra,dec and redshift
print("Loading Catalog Data:")
ra = np.array(f['catalog/gold/ra'])[np.array(f['index/select'])].copy() #'index/select' ensures appropriate shear measurment
dec = np.array(f['catalog/gold/dec'])[np.array(f['index/select'])].copy()

z = np.array(f['catalog/bpz/unsheared/zmean_sof'])[np.array(f['index/select'])].copy()
z_mc = np.array(f['catalog/bpz/unsheared/zmc_sof'])[np.array(f['index/select'])].copy()

print("Computing Comoving Distance:")
dist = cosmo.comoving_distance(z)
dist_mc = cosmo.comoving_distance(z_mc)

# Remove non-applicable data points identified by negative redshift 
ind_z = np.where(z>0)[0]
ra = ra[ind_z]
dec = dec[ind_z]
z = z[ind_z]
z_mc = z_mc[ind_z]
dist = dist[ind_z]
dist_mc = dist_mc[ind_z]

# Load shear data
print("Loading Shear Data:")
dgamma = 2*0.01
R = 0.5*(np.array(f['catalog/metacal/unsheared/R11'])[np.array(f['index/select'])][indz]+np.array(f['catalog/metacal/unsheared/R22'])[np.array(f['index/select'])][indz]).copy()
Rs = 0.0120708305447 #selection response

e1 = np.array(f['catalog/metacal/unsheared/e_1'])[np.array(f['index/select'])][indz].copy()
e2 = np.array(f['catalog/metacal/unsheared/e_2'])[np.array(f['index/select'])][indz].copy()

In [8]:
# Remove non-applicable data points identified by negative redshift 
ind_z = np.where(z>0)[0]
ra = ra[ind_z]
dec = dec[ind_z]
z = z[ind_z]
z_mc = z_mc[ind_z]
dist = dist[ind_z]
dist_mc = dist_mc[ind_z]

# Load shear data
print("Loading Shear Data:")
dgamma = 2*0.01
R = 0.5*(np.array(f['catalog/metacal/unsheared/R11'])[np.array(f['index/select'])][indz]+np.array(f['catalog/metacal/unsheared/R22'])[np.array(f['index/select'])][indz]).copy()
Rs = 0.0120708305447 #selection response

e1 = np.array(f['catalog/metacal/unsheared/e_1'])[np.array(f['index/select'])][indz].copy()
e2 = np.array(f['catalog/metacal/unsheared/e_2'])[np.array(f['index/select'])][indz].copy()

Loading Shear Data:


In [40]:
# km = kmeans_radec.KMeans(np.loadtxt('kmeans_centers_npix100_desy3.dat')) ### the coordinates for the centers of JK patched: pre-calculated
## or you can define your own "km": how-to is on the documentation -- super simple
## see the kmeans_radec documetation how to do it

def GenerateJKRegions(ra, dec, njack, jfile, maxiter=200, tol=1.0e-5):
    """
    Generate k-means clusters from a set of data, using `kmeans_radec <https://github.com/esheldon/kmeans_radec>`_.
    For roughly uniform data, it generates N-clusters of roughly equal cardinality.
    Here, distances are computed on the surface of the unit sphere, and coordinates are given as RA/DEC.

    Parameters
    ----------
    ra (float array)
        Right ascension values for each data point.
    dec (float array)
        Declination values for each data point.
    njack (int)
        Number of k-means clusters to generate, i.e. the number of JK regions.
    jfile (str)
        Output file name to save the regions.
    maxiter (int)
        Maximum number of iterations for the k-means generation, see `kmeans_radec documentation <https://github.com/esheldon/kmeans_radec>`_.
    tol (float)
        Tolerance level needed to be considered converged, see `kmeans_radec documentation <https://github.com/esheldon/kmeans_radec>`_.

    Returns
    -------
    None

    """

    rd = np.zeros((len(ra), 2))
    rd[:, 0] = ra
    rd[:, 1] = dec

    km = kmeans_radec.kmeans_sample(rd, njack, maxiter=maxiter, tol=tol)

    if not km.converged:
        raise RuntimeError("k means did not converge")

#     dir = os.path.dirname(jfile)
#     if not os.path.exists(dir):
#         os.makedirs(dir)

    return km.centers
    
n_jack = 100
jk_file_str = "kmeans_centers_npix100_desy3.dat"
km = GenerateJKRegions(ra, dec, n_jack, jk_file_str)

X (20020, 2)  centers (100, 2)  tol=1e-05  maxiter=200
41 iterations  cluster sizes: [209 176 160 272 164 204 228 234 176 227 214 237 126 243 205 209 223 262
 243 280 259 261 189 156 207 156 165 205 148 158 243 263 231 260 186 215
 235 133 188 189 180 168 201 166 282 184 189 154 188 210 144 192 186 272
 202 156 157 212 262 148 251 145 164 214 248 184 340 190 184 169 153 175
 245 217 203 235 161 224 198 145 172 211 168 182 276 196 167 174 162 209
 235 152 223 184 222 191 195 160 142 162]
X (100208944, 2)  centers (100, 2)  tol=1e-05  maxiter=200


MemoryError: Unable to allocate 74.7 GiB for an array with shape (100208944, 100) and data type float64

In [ ]:
# Convert right ascension to take into account DES footprint
ra_tem = ra.copy()
ra_tem[ra_tem > 250.] -= 360.
ra_dec = np.vstack((ra_tem,dec)).T
jk = km.find_nearest(ra_dec)  #### this will assign each objects a jk number


### how to select galaxies around the JK patches
#center data = jk patch you calculated
pos_jk = astropy.SkyCoord(ra=center_data[0]*u.deg,dec=center_data[1]*u.deg) ## position of the jk centers
jk_dist = pos_jk[jkid].separation(pos_jk).degree ### the distance to each JK center from each JK center
ind = jk_dist < 20.8 ### maximum angular scale to include: you can modify this number based on the maximum radius of interest and the size of the JK patches
jk_neigh = np.arange(100)[ind] ### the indices of JK patched to include
ind_jk_src = np.in1d(jk,jk_neigh) ### selected galaxies that have the above JK indices

ra = ra[ind_jk_src]
dec = dec[ind_jk_src]
z = z[ind_jk_src]
z_mc = z_mc[ind_jk_src]
dist = dist[ind_jk_src]
dist_mc = dist_mc[ind_jk_src]
R = R[ind_jk_src]
e1 = e1[ind_jk_src]
e2 = e2[ind_jk_src]

# if you need additional filter, do similarly. 

#Then you Load cluster data....
#and the random data...
#select those are included in our JK patch..

#### you can for now use "redmapper" clusters that are already in the master catalog above.
#### try to find it, and test your code. 
#### there are also random catalogs for the redmapper which you can ignore for now

# do Treecorr.
### the lines from the last time starts.

In [ ]:
NZBIN ## number of redshift bins
NBINS ## number of radial bins
top = np.zeros((NZBIN,NBINS))
top_im = np.zeros((NZBIN,NBINS))
bottom = np.zeros((NZBIN,NBINS))
weight = np.zeros((NZBIN,NBINS))
for i in range(len(NZBINS))
    #select clusters in this redshift bins
    cat_cl = treecorr.Catalog(ra=RA_cl_zbin,dec=dec_cl_zbin,ra_units='deg',dec_units='deg')
    z_cl_zbin = np.mean(z_cl[_selection_])
    dist_zbin = (comoving_distance(z_cl_zbin).value*10**6)*0.7
    wt_gal = (1./(1.663*10**12))*(1+z_cl_zbin)*dist_zbin*(1.-dist_zbin/distance_to_the_galaxies_from_zmean)
    sci = (1./(1.663*10**12))*(1.+z_cl_zbin)*dist_zbin*(1.-dist_zbin/dist_to_the_galaxies_from_zmc)
    sci[sci<0.] = 0. ###### important
    gal_sel = (zmean_of_galaxies>=z_cl_zbin+0.1) #_cushion_between_clusters_and_galaxies
    cat_gal = treecorr.Catalog(g1=gamma1[gal_sel],g2=gamma2[gal_sel],ra=ra_gal[...],dec=dec_gal[...],w=wt[...],ra_units='deg',dec_units='deg')
    cat_gal2 = treecorr.Catalog(g1=gamma1[gal_sel],g2=gamma2[gal_sel],ra=ra_gal[...],dec=dec_gal[...],w=(wt*sci*(R+Rs))[...],ra_units='deg',dec_units='deg')
    thmin = np.arctan(Rmin*10**6 / dist_cl_zbin) * (180./np.pi) * 60. ### minimum angular distance in arcmin
    thmin = np.arctan(Rmax*10**6 / dist_cl_zbin) * (180./np.pi) * 60.
    ng = treecorr.NGCorrelation(nbins=NBINS,min_sep=thmin,max_sep=thmax,sep_units='arcmin',verbose=2,bin_slop=0.02) ## confinguration of the NGcorrelation :: not a real calculation yet.
    ng2 = treecorr.NGCorrelation(nbins=NBINS,min_sep=thmin,max_sep=thmax,sep_units='arcmin',verbose=2,bin_slop=0.02)
    ng.process(cat_cl,cat_gal) ### actual calcalation
    ng2.process(cat_cl,cat_gal2)
    top[i] = ng.xi*ng.weight #### remove the denominator part so that we are only left with the numerator
    top_im[i] = ng.xi_im*ng.weight
    bottom[i] = ng2.weight #### we selected the denominator part with the different weight (R*sci*wt)
    weight[i] = ng.weight #### for calculating boost factor in the future

### do it for every JK patch and combine

top_total = sum(top)
bottom_total = sum(bottom)

## and then
top_total/bottom_total

### do this for 100 JK realization and calculate JK cov.

test
